In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
kf = KFold(n_splits=3, shuffle=True)

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
combi = pd.concat([train.drop('author', axis=1), test], axis=0)

train = train.sample(frac=1.0) # shuffle

x = train.text
y = train.author

In [95]:
from sklearn.dummy import DummyClassifier

dum = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classify', DummyClassifier())])

cv = cross_val_score(dum, x, y, cv=4, scoring='accuracy')

print('Mean score:', cv.mean())
print('Std Dev:   ', cv.std())

Mean score: 0.341437024678
Std Dev:    0.00588007889459


In [6]:
pl = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classify', MultinomialNB())
])

param_grid = [
    {
        'tfidf__max_df': np.arange(.01,.10,.01),
        'tfidf__min_df': [2,3,4],
        'tfidf__ngram_range': [(1,2)],
        'tfidf__norm': ['l2'],
        'classify__alpha': [.001, .01, .1],
    },
]

grid =\
GridSearchCV(pl, cv=3, n_jobs=-1, param_grid=param_grid, scoring='accuracy')\
.fit(x, y)

model = grid.best_estimator_
print(model)
cv = cross_val_score(model, x, y, cv=4, scoring='accuracy')

print('Mean score:', cv.mean())
print('Std Dev:   ', cv.std())

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.089999999999999997, max_features=None,
        min_df=2, ngram_range=(1, 2), norm='l2', preprocessor=None,
 ...f=True, vocabulary=None)), ('classify', MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True))])
Mean score: 0.851422370809
Std Dev:    0.00198274978384


# Convolutional Neural Network

In [14]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

# set parameters:
max_features = 20000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 10

In [15]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.array(pd.get_dummies(y))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 25943 unique tokens.
Shape of data tensor: (19579, 400)
Shape of label tensor: (19579, 3)


In [16]:
x_train, x_test, y_train, y_test =\
train_test_split(data, labels, test_size=.2)

In [17]:
filters = 250
kernel_size = 3
hidden_dims = 250

print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

es = EarlyStopping(min_delta=.01, patience=2)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs, callbacks=[es],
          validation_data=(x_test, y_test))

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 400, 50)           1000000   
_________________________________________________________________
dropout_6 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 398, 250)          37750     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 250)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_7 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_9 (Activation)    (None, 250)               0     

In [18]:
filters = 250
kernel_size = 5
hidden_dims = 250

print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 5,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

es = EarlyStopping(min_delta=.01, patience=2)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs, callbacks=[es],
          validation_data=(x_test, y_test))

Build model...
Train on 15663 samples, validate on 3916 samples
Epoch 1/10
15663/15663 [==============================] - 306s - loss: 0.8109 - acc: 0.6295 - val_loss: 0.6217 - val_acc: 0.7467

In [20]:
filters = 250
kernel_size = 5
hidden_dims = 250

print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

es = EarlyStopping(min_delta=.01, patience=2)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs, callbacks=[es],
          validation_data=(x_test, y_test))

Build model...
Train on 15663 samples, validate on 3916 samples
Epoch 1/10
15663/15663 [==============================] - 281s - loss: 0.7430 - acc: 0.6554 - val_loss: 0.4955 - val_acc: 0.8008

In [ ]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

# Recurrent Neural Network

In [21]:
from keras.layers import LSTM

In [24]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15, callbacks=[es],
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Build model...
Train...
Train on 15663 samples, validate on 3916 samples
Epoch 1/15
15663/15663 [==============================] - 674s - loss: 0.7034 - acc: 0.6910 - val_loss: 0.5105 - val_acc: 0.8008

In [ ]:
# TODO:
# Ensemble the 3 models